In [31]:
import os
import glob
import shutil
import pydicom

In [32]:
HOMEPATH = os.getenv('HOME')
DATAPATH = os.path.join(HOMEPATH, 'Data', 'testfMRIBreastData')

In [33]:
SRCFLDR = os.path.join(DATAPATH, 'rawS3')
DSTFLDR = os.path.join(DATAPATH, 'StudyData')
FEXT = 'dcm'

print(f'Working directory is {SRCFLDR}')
print(f'Saving directory is {DSTFLDR}')

Working directory is /Users/joseulloa/Data/testfMRIBreastData/rawS3
Saving directory is /Users/joseulloa/Data/testfMRIBreastData/StudyData


In [34]:
dcmlist = glob.glob(os.path.join(SRCFLDR,f'*.{FEXT}'))
print(f'There are {len(dcmlist)} files in {SRCFLDR}')

There are 12600 files in /Users/joseulloa/Data/testfMRIBreastData/rawS3


In [39]:
# Sort out dicom files following this order:
# PatientID ()
# PatientName ()
# StudyID ()
# StudyDate ()
# SeriesNro ()
# SeriesDescription ()
os.makedirs(DSTFLDR, exist_ok=True)
for dcm in dcmlist:
    ds = pydicom.dcmread(dcm,stop_before_pixels=True)
    # if (0x0018,0x0010) in ds:
    #     SubSeries = ds.ContrastBolusAgent
    #     if not SubSeries:
    #         SubSeries = 'NoContrast'
    # else:
    #     SubSeries = 'NoContrast'
    [PatientID, StudyID, StudyDate, SeriesNro, TempPos] = [ds.PatientID, 
                                                           ds.StudyID, 
                                                           ds.StudyDate, 
                                                           str(ds.SeriesNumber), 
                                                           str(ds.TemporalPositionIdentifier)]
    folderStruct = os.path.join(DSTFLDR, 
                                PatientID.replace(' ','_'), 
                                StudyID.replace(' ','_'), 
                                StudyDate.replace(' ','_'), 
                                SeriesNro.replace(' ','_'), 
                                # SubSeries.replace(' ','_'),
                                TempPos.replace(' ','_'))
    os.makedirs(folderStruct, exist_ok=True)
    dstFile = os.path.join(folderStruct, os.path.split(dcm)[-1])
    if not os.path.isfile(dstFile):
        shutil.copy2(dcm, dstFile)
